<a href="https://colab.research.google.com/github/fpcaforio/grace/blob/master/Grad_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pickle
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import cv2

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#CONFIG

dataset = 'KDD-CUP99'

if dataset == 'NLS-KDD':
  print('Dataset: NLS-KDD')
  pathMagneto = '/content/drive/MyDrive/Grace/NLS-KDD/magneto/'
  size = 12
elif dataset == 'UNSW-NB15':
  print('Dataset: UNSW-NB15')
  pathMagneto = '/content/drive/MyDrive/Grace/UNSW-NB15/magneto/'
  size = 15
elif dataset == 'KDD-CUP99':  
  print('KDD-CUP99')
  pathMagneto = '/content/drive/MyDrive/Grace/KDD-CUP99/magneto/'
  size = 13

N_CLASSES = 2

KDD-CUP99


In [7]:
model = tf.keras.models.load_model(pathMagneto + 'CNN_model.h5')
model.summary()

last_conv_layer_name = 'conv2'

# all layers
classifier_layer_names = [
    'flatten', 'dense', 'dense_1', 'dense_2',
]

with open(pathMagneto + 'train_image.pickle', 'rb') as t:
    XTrain = pickle.load(t)
t.close

with open(pathMagneto + 'test_image.pickle', 'rb') as t:
    XTest = pickle.load(t)
t.close

with open(pathMagneto + 'Ytrain.pickle', 'rb') as k:
    YTrain = pickle.load(k)
k.close

with open(pathMagneto + 'Ytest.pickle', 'rb') as k:
    YTest = pickle.load(k)
k.close

XTrain = np.array(XTrain)
XTest = np.array(XTest)
YTrain = np.array(YTrain)
YTest = np.array(YTest)

print('XTrain: ', XTrain.shape)
print('XTest: ',XTest.shape)
print('YTrain: ',YTrain.shape)
print('YTest: ',YTest.shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13, 13, 3)]       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 11, 11, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 9, 9, 64)          18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 9, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0     

In [8]:
imgrgb = np.stack((XTest,)*3, axis=-1)
preds = model.predict(imgrgb, verbose=1)
y_pred = np.argmax(preds, axis=1)
con_mat = confusion_matrix(YTest, y_pred)

tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print(r)

9720/9720 [==============================] - 120s 12ms/step
(230777, 19659, 538, 60055, 0.9350639329451594, 0.9563109945101649, 0.9976741672610941, 0.9215009024261688, 0.9580758524632019, 0.008878913405838958, 0.9215009024261688)


In [9]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):

    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)

    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()

    for i in range(pooled_grads.shape[-1]):  # 128
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

    return heatmap

In [14]:
imgrgb = np.stack((XTrain,)*3, axis=-1)

XTrain_heatmap = []

for i in range (0, len(XTrain)):
  
  print('Istanza #: ', i)

  array = np.expand_dims(imgrgb[i], axis=0)
  heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name, classifier_layer_names)
  heatmap = cv2.resize(heatmap, (size, size))
  #plt.imshow(heatmap)
  #plt.show()
  XTrain_heatmap.append(heatmap)

#f_myfile = open(pathSave, 'wb') 
#pickle.dump(XTrain_heatmap, f_myfile)
#f_myfile.close()

Istanza #:  0
Istanza #:  1
Istanza #:  2
Istanza #:  3
Istanza #:  4
Istanza #:  5
Istanza #:  6
Istanza #:  7
Istanza #:  8
Istanza #:  9
Istanza #:  10
Istanza #:  11
Istanza #:  12
Istanza #:  13
Istanza #:  14
Istanza #:  15
Istanza #:  16
Istanza #:  17
Istanza #:  18
Istanza #:  19
Istanza #:  20
Istanza #:  21
Istanza #:  22
Istanza #:  23
Istanza #:  24
Istanza #:  25
Istanza #:  26
Istanza #:  27
Istanza #:  28
Istanza #:  29
Istanza #:  30
Istanza #:  31
Istanza #:  32
Istanza #:  33
Istanza #:  34
Istanza #:  35
Istanza #:  36
Istanza #:  37
Istanza #:  38
Istanza #:  39
Istanza #:  40
Istanza #:  41
Istanza #:  42
Istanza #:  43
Istanza #:  44
Istanza #:  45
Istanza #:  46
Istanza #:  47
Istanza #:  48
Istanza #:  49
Istanza #:  50
Istanza #:  51
Istanza #:  52
Istanza #:  53
Istanza #:  54
Istanza #:  55
Istanza #:  56
Istanza #:  57
Istanza #:  58
Istanza #:  59
Istanza #:  60
Istanza #:  61
Istanza #:  62
Istanza #:  63
Istanza #:  64
Istanza #:  65
Istanza #:  66
Istan

Exception ignored in: <function ScopedTFGraph.__del__ at 0x7f006c5279e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
KeyboardInterrupt


Istanza #:  68
Istanza #:  69
Istanza #:  70
Istanza #:  71
Istanza #:  72
Istanza #:  73
Istanza #:  74
Istanza #:  75
Istanza #:  76
Istanza #:  77
Istanza #:  78
Istanza #:  79
Istanza #:  80
Istanza #:  81
Istanza #:  82
Istanza #:  83
Istanza #:  84
Istanza #:  85
Istanza #:  86
Istanza #:  87
Istanza #:  88
Istanza #:  89
Istanza #:  90
Istanza #:  91
Istanza #:  92
Istanza #:  93
Istanza #:  94
Istanza #:  95
Istanza #:  96
Istanza #:  97
Istanza #:  98
Istanza #:  99
Istanza #:  100
Istanza #:  101
Istanza #:  102
Istanza #:  103
Istanza #:  104
Istanza #:  105
Istanza #:  106
Istanza #:  107
Istanza #:  108
Istanza #:  109
Istanza #:  110
Istanza #:  111
Istanza #:  112
Istanza #:  113
Istanza #:  114
Istanza #:  115
Istanza #:  116
Istanza #:  117
Istanza #:  118
Istanza #:  119
Istanza #:  120
Istanza #:  121
Istanza #:  122
Istanza #:  123
Istanza #:  124
Istanza #:  125
Istanza #:  126
Istanza #:  127
Istanza #:  128
Istanza #:  129
Istanza #:  130
Istanza #:  131
Istanza 

KeyboardInterrupt: ignored

In [15]:
imgrgb = np.stack((XTest,)*3, axis=-1)

XTest_heatmap = []

for i in range (0, len(XTest)):

  print('Istanza #: ', i)

  array = np.expand_dims(imgrgb[i], axis=0)
  heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name, classifier_layer_names)
  heatmap = cv2.resize(heatmap, (size, size))
  #plt.imshow(heatmap)
  #plt.show()
  XTest_heatmap.append(heatmap)

#f_myfile = open(pathSave, 'wb') 
#pickle.dump(XTrain_heatmap, f_myfile)
#f_myfile.close()

Istanza #:  0
Istanza #:  1
Istanza #:  2
Istanza #:  3
Istanza #:  4
Istanza #:  5
Istanza #:  6
Istanza #:  7
Istanza #:  8
Istanza #:  9
Istanza #:  10
Istanza #:  11
Istanza #:  12
Istanza #:  13
Istanza #:  14
Istanza #:  15
Istanza #:  16
Istanza #:  17
Istanza #:  18
Istanza #:  19
Istanza #:  20
Istanza #:  21
Istanza #:  22
Istanza #:  23
Istanza #:  24
Istanza #:  25
Istanza #:  26
Istanza #:  27
Istanza #:  28
Istanza #:  29
Istanza #:  30
Istanza #:  31
Istanza #:  32
Istanza #:  33
Istanza #:  34
Istanza #:  35
Istanza #:  36
Istanza #:  37
Istanza #:  38
Istanza #:  39
Istanza #:  40
Istanza #:  41
Istanza #:  42
Istanza #:  43
Istanza #:  44
Istanza #:  45
Istanza #:  46
Istanza #:  47
Istanza #:  48
Istanza #:  49
Istanza #:  50
Istanza #:  51
Istanza #:  52
Istanza #:  53
Istanza #:  54
Istanza #:  55
Istanza #:  56
Istanza #:  57
Istanza #:  58
Istanza #:  59
Istanza #:  60
Istanza #:  61
Istanza #:  62
Istanza #:  63
Istanza #:  64
Istanza #:  65
Istanza #:  66
Istan

Exception ignored in: <function ScopedTFGraph.__del__ at 0x7f006c5279e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
KeyboardInterrupt


Istanza #:  131
Istanza #:  132
Istanza #:  133
Istanza #:  134
Istanza #:  135
Istanza #:  136
Istanza #:  137
Istanza #:  138
Istanza #:  139
Istanza #:  140
Istanza #:  141
Istanza #:  142
Istanza #:  143
Istanza #:  144
Istanza #:  145
Istanza #:  146
Istanza #:  147
Istanza #:  148
Istanza #:  149
Istanza #:  150
Istanza #:  151
Istanza #:  152
Istanza #:  153
Istanza #:  154
Istanza #:  155
Istanza #:  156
Istanza #:  157
Istanza #:  158
Istanza #:  159
Istanza #:  160
Istanza #:  161
Istanza #:  162
Istanza #:  163
Istanza #:  164
Istanza #:  165
Istanza #:  166
Istanza #:  167
Istanza #:  168
Istanza #:  169
Istanza #:  170
Istanza #:  171
Istanza #:  172
Istanza #:  173
Istanza #:  174
Istanza #:  175
Istanza #:  176
Istanza #:  177
Istanza #:  178
Istanza #:  179
Istanza #:  180
Istanza #:  181
Istanza #:  182
Istanza #:  183
Istanza #:  184
Istanza #:  185
Istanza #:  186
Istanza #:  187
Istanza #:  188
Istanza #:  189
Istanza #:  190
Istanza #:  191
Istanza #:  192
Istanza 

KeyboardInterrupt: ignored